# Pansharpening GANs -- training set creation

## Setup

In [1]:
import os
import pandas as pd
import numpy as np

### Process raw rasters

In [ ]:
# read all files
raster_files = [scn for scn in os.listdir('raw_scenes') if scn.split('.')[-1] == '.tif']

# separate by panchromatic and multispectral
raster_pan = [scn in raster_files if 'P1BS' in scn]
raster_ms = [scn in raster_files if 'M1BS' in scn]

# process scenes
for scn_pan in raster_pan:
    scn_ms = f"{scn.split('-')[0]}M1BS{scn.split('-')[2]}"
    # check if matching multispectral scene is present
    if scn_ms in raster_ms:
        !python generate_training_scene.py --pan_scene=$scn_pan \
                                           --ms_scene=$scn_ms \
                                           --output_foler="processed_scenes"

### Tile processed rasters

In [2]:
# find all panchromatic processed rasters
processed_pan = [scn for scn in os.listdir('processed_scenes') if 'pan' in scn]

# tile raster pairs (split into training and validation)
prob_val = 0.1
for scn_pan in processed_pan:
    if np.random.random() > prob_val:
        !python tile_training_scene.py --file_pan=$scn_pan --dataset="training" \
                                       --patch_size=336 --stride=0.7
    else:
        !python tile_training_scene.py --file_pan=$scn_pan --dataset="validation" \
                                       --patch_size=224 --stride=1